In [1]:
import os

In [2]:
print(os.getcwd())

C:\Users\rgarcia\Documents\code


In [3]:
import pandas as pd

In [4]:
from urllib.parse import urlparse

In [5]:
from IPython.display import display, Markdown

In [6]:
UF_list = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MS', 'MT', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 
           'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']


In [7]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/600.1.17\
 (KHTML, like Gecko) Version/8.0 Safari/600.1.17'

In [8]:
consulta_cand = 'https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/'

In [9]:
os.getcwd()

'C:\\Users\\rgarcia\\Documents\\code'

In [11]:
os.chdir('C:\\Users\\rgarcia\\Documents\\data')

In [12]:
os.getcwd()

'C:\\Users\\rgarcia\\Documents\\data'

In [13]:
for i in UF_list:
    os.system(f'wget --header="User-Agent: {user_agent}" {consulta_cand}rede_social_candidato_2022_{i}.zip')
    os.system(f'unzip -p rede_social_candidato_2022_{i}.zip rede_social_candidato_2022_{i}.csv > SocNet_{i}.csv')
    os.system(f'del rede_social_candidato_2022_{i}.zip')
    

In [14]:
all_files = [i for i in os.listdir() if i[0:6] == 'SocNet']

In [15]:
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep=';', encoding='ISO-8859-1', decimal=',', low_memory=False)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [16]:
for i in all_files:
    os.system(f'del {i}')

In [17]:
frame.shape ; frame.to_csv('candidato_redes_total.csv', index=False)

In [18]:
frame['domain'] = frame['DS_URL'].apply(lambda x: urlparse(x).netloc)

In [19]:
frame['domain'] = frame['domain'].str.lower()

In [20]:
frame['domain'] = frame.domain.str.replace('www.', '')

C:\Users\rgarcia\AppData\Local\Temp\ipykernel_6760\4068980154.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  frame['domain'] = frame.domain.str.replace('www.', '')


In [21]:
df = frame[['SQ_CANDIDATO', 'NR_ORDEM', 'DS_URL', 'domain']].sort_values(by=['SQ_CANDIDATO', 'NR_ORDEM'])

In [22]:
df.DS_URL = df.DS_URL.str.lower()

In [23]:
def md(x):
     display(Markdown(x))

In [24]:
md(f'> ### Quantidade de sites/perfis de rede registrados: \n > # <u>{frame.shape[0]}</u> \n')

> ### Quantidade de sites/perfis de rede registrados: 
 > # <u>43716</u> 


In [25]:
# Filtros para identificação das redes

In [26]:
facebook_id = ['facebook', 'faceoo', 'fecebo', 'feicib', 'fb.me', 'fb.com', 'acebook', 
               'faacebook', 'facebbook', 'fb.audio', 'fb.gg', 'fb.me', 'fb.watch', 'facebbok',
               'faceboo', 'facebok']

In [27]:
twitter_id = ['twitter', 'twiter', 'tweet', 'twiit']

In [28]:
instagram_id = ['instagram', 'instrag', 'istagra', 'istragra', 'instragam', 'instragram',
                'intagram', 'intragra', 'instagran', 'tagran', 'tragran', 'instragran',
                'instagam', 'instgram', 'istragam', 'www.instagram', 'www.instagram.com', 'insta',
                'insagram']

In [29]:
youtube_id = ['youtube', 'yotube', 'yutub', 'outube', 'youtobe', 'youtu.be']

In [30]:
tiktok_id = ['tiktok', 'tictoc', 'tictok', 'tiktoc', 'iktok', 'iktock', 'ictock', 'tktok',
             'tiktk', 'tiktik', 'tiktak', 'Douyin']

In [31]:
gettr_id = ['gettr']; snapchat_id = ['snapchat'] ; blogger_id = ['blogger']
kwai_id = ['kwai', 'kw.ai', 'kuaishou'] ; telegram_id = ['telegram', 't.me'] ; linkedin_id = ['linkedin']
pinterest_id = ['pinterest'] ; flickr_id = ['flickr'] ; whatsapp_id = ['whatsapp', 'wa.me']

In [32]:
id_labels = [facebook_id, twitter_id, instagram_id, youtube_id, tiktok_id,
             gettr_id, snapchat_id, blogger_id, kwai_id, telegram_id, linkedin_id,
             pinterest_id, flickr_id, whatsapp_id, ['spotify'], ['wechat', 'weixin'],
             ['reddit'], ['quora'], ['github'], ['tencent', 'qq.com'], ['twitch'],
             ['discord'], ['parler'], ['soundcloud'], ['sina weibo', 'weibo'],
             ['facebook messenger', 'messenger.com'], ['skype'], ['4chan'],
             ['gab.com'], ['linktree', 'linktr.ee'], ['myspace'], ['tumblr'],
             ['wordpress'], ['wixsite'], ['wikipedia'], ['tinder'], ['last.fm'], ['deezer'],
            ['triller'], ['periscope'], ['vimeo'], ['baidu'], ['badoo']]

In [33]:
# Função para classificar as redes

def id_network(x):
    
    net_id = ''

    for j in id_labels:
        if any(i in x for i in j):
            net_id = j[0].title() 
        else:
            pass

    if net_id == '':
        net_id = 'Outros'

    return net_id

In [34]:
df['rede_social'] = df.DS_URL.apply(id_network)

In [35]:
valida_redes = df[['SQ_CANDIDATO',
                          'domain', 'rede_social']].drop_duplicates().groupby(['domain', 'rede_social'
                          ]).SQ_CANDIDATO.count().reset_index().sort_values(by='SQ_CANDIDATO', ascending=False)

In [36]:
valida_redes.head(40)

,domain,rede_social,SQ_CANDIDATO
2006,instagram.com,Instagram,13082
1568,facebook.com,Facebook,11153
3986,twitter.com,Twitter,3196
4216,youtube.com,Youtube,2365
3924,tiktok.com,Tiktok,2070
2448,linkedin.com,Linkedin,509
2574,m.facebook.com,Facebook,338
3338,pt-br.facebook.com,Facebook,292
4144,web.facebook.com,Facebook,291
3813,t.me,Telegram,187


In [37]:
valida_redes_rank = valida_redes.groupby(['rede_social']).SQ_CANDIDATO.sum().reset_index().sort_values(by='SQ_CANDIDATO', ascending=False).reset_index(drop=True)

In [38]:
print(valida_redes_rank)

           rede_social  SQ_CANDIDATO
0            Instagram         13419
1             Facebook         12311
2               Outros          4206
3              Twitter          3452
4              Youtube          2431
5               Tiktok          2263
6             Linkedin           555
7             Telegram           218
8                 Kwai           168
9             Linktree           138
10            Whatsapp           125
11              Flickr            63
12               Gettr            57
13             Spotify            25
14             Wixsite            24
15           Pinterest            10
16            Snapchat             8
17           Wordpress             7
18          Soundcloud             6
19              Parler             6
20              Twitch             4
21           Wikipedia             4
22             Blogger             2
23              Tinder             2
24              Tumblr             2
25  Facebook Messenger             2
2

In [39]:
# Os candidatos com Tinder

df[(df.SQ_CANDIDATO == 120001650955) | (df.SQ_CANDIDATO == 130001615155)]

,SQ_CANDIDATO,NR_ORDEM,DS_URL,domain,rede_social
13155,120001650955,1,https://twitter.com/andrelacerdams,twitter.com,Twitter
13153,120001650955,2,https://www.tiktok.com/@andrelacerdams?_t=8uhp...,tiktok.com,Tiktok
13156,120001650955,3,https://tinder.com/@andrelacerdams,tinder.com,Tinder
13157,120001650955,4,https://www.linkedin.com/mwlite/in/andrelacerdams,linkedin.com,Linkedin
13154,120001650955,5,https://www.facebook.com/andrelacerdams,facebook.com,Facebook
13152,120001650955,6,https://www.instagram.com/andrelacerdams,instagram.com,Instagram
11543,130001615155,1,https://www.instagram.com/mercio.m.antunes/,instagram.com,Instagram
11544,130001615155,2,https://twitter.com/merciomotamg,twitter.com,Twitter
11545,130001615155,3,https://www.facebook.com/profile.php?id=100084...,facebook.com,Facebook
11542,130001615155,4,https://www.youtube.com/channel/uc2fkkbb-logun...,youtube.com,Youtube


In [40]:
df_dd = df[['SQ_CANDIDATO', 'rede_social']].reset_index(drop=True)

In [41]:
md(f'> ### Total de candidatos com pelo menos uma rede/site: \n > # {str(len(df.SQ_CANDIDATO.unique()))}')

> ### Total de candidatos com pelo menos uma rede/site: 
 > # 15544

In [47]:
print(str(round(((15409/29259)*100),1)) + ' %')

52.7 %


In [48]:
ranking_redes = df_dd.groupby('rede_social').SQ_CANDIDATO.count().reset_index().sort_values(by='SQ_CANDIDATO', ascending=False).reset_index(drop=True)

In [49]:
ranking_outros = ranking_redes[ranking_redes.rede_social == 'Outros'].reset_index(drop=True)

In [50]:
ranking_com_id = ranking_redes[ranking_redes.rede_social != 'Outros'].reset_index(drop=True)

In [51]:
ranking_redes = pd.concat([ranking_com_id, ranking_outros]).reset_index(drop=True)

In [52]:
del ranking_com_id ; del ranking_outros

In [66]:
print(valida_redes_rank)

      Rede Social/Site  Registros de candidatos
0            Instagram                    13419
1             Facebook                    12311
2               Outros                     4206
3              Twitter                     3452
4              Youtube                     2431
5               Tiktok                     2263
6             Linkedin                      555
7             Telegram                      218
8                 Kwai                      168
9             Linktree                      138
10            Whatsapp                      125
11              Flickr                       63
12               Gettr                       57
13             Spotify                       25
14             Wixsite                       24
15           Pinterest                       10
16            Snapchat                        8
17           Wordpress                        7
18          Soundcloud                        6
19              Parler                  

In [54]:
twiteiros = df[df.rede_social == 'Twitter']

In [55]:
twiteiros['handle'] = twiteiros.DS_URL.apply(lambda x: x.split('.com/')[-1].split("?")[0].replace(':','').replace('@','').split("/")[0])

C:\Users\rgarcia\AppData\Local\Temp\ipykernel_6760\4169020061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twiteiros['handle'] = twiteiros.DS_URL.apply(lambda x: x.split('.com/')[-1].split("?")[0].replace(':','').replace('@','').split("/")[0])


In [56]:
twiteiros

,SQ_CANDIDATO,NR_ORDEM,DS_URL,domain,rede_social,handle
330,10001601430,3,https://twitter.com/eduribeiroac,twitter.com,Twitter,eduribeiroac
213,10001606307,3,https://twitter.com/odavidhall36,twitter.com,Twitter,odavidhall36
123,10001606585,2,https://twitter.com/dimassandas,twitter.com,Twitter,dimassandas
133,10001613550,3,https://twitter.com/leodebritoac,twitter.com,Twitter,leodebritoac
9,10001622769,3,https://twitter.com/dramichellemelo,twitter.com,Twitter,dramichellemelo
...,...,...,...,...,...,...
43353,270001697127,2,https://twitter.com/irajasenador?t=u3e5yrd3lkd...,twitter.com,Twitter,irajasenador
43586,270001700073,2,https://twitter.com/tiaodaelka?s=11&t=o6zi-ovk...,twitter.com,Twitter,tiaodaelka
43490,270001700076,3,https://twitter.com/deputadoelenil?s=11&t=mhp_...,twitter.com,Twitter,deputadoelenil
43368,270001713170,3,https://twitter.com/katiaabreu,twitter.com,Twitter,katiaabreu


In [57]:
facebookeiros = df[df.rede_social == 'Facebook'].reset_index(drop=True)

In [58]:
facebookeiros

,SQ_CANDIDATO,NR_ORDEM,DS_URL,domain,rede_social
0,10001601425,2,https://www.facebook.com/driellemendes.pereira,facebook.com,Facebook
1,10001601427,2,https://www.facebook.com/pablobregense,facebook.com,Facebook
2,10001601430,2,https://www.facebook.com/eduribeiroac/,facebook.com,Facebook
3,10001601444,2,https://www.facebook.com/cleo.meneses.9,facebook.com,Facebook
4,10001601615,3,https://www.facebook.com/vandamilani.ac,facebook.com,Facebook
...,...,...,...,...,...
15078,270001700731,1,https://www.facebook.com/simone.alves.54772,facebook.com,Facebook
15079,270001700732,2,https://www.facebook.com/josyanne.fenelon.7,facebook.com,Facebook
15080,270001713170,2,https://www.facebook.com/senadorakatiaoficial,facebook.com,Facebook
15081,270001725829,1,https://www.facebook.com/profile.php?id=100010...,facebook.com,Facebook


In [59]:
df[df.rede_social == 'Whatsapp']

,SQ_CANDIDATO,NR_ORDEM,DS_URL,domain,rede_social
252,10001639519,8,https://chat.whatsapp.com/ibvuca8u5qeepgdkpvkwa4,chat.whatsapp.com,Whatsapp
198,10001649256,8,https://api.whatsapp.com/send?phone=11999023378,api.whatsapp.com,Whatsapp
1618,40001609310,8,https://wa.me/message/ocweirm5hwkye1,wa.me,Whatsapp
1212,40001609314,5,https://wa.me/qr/bfa6wf5j2hw4l,wa.me,Whatsapp
1284,40001609326,1,https://web.whatsapp.com/,web.whatsapp.com,Whatsapp
...,...,...,...,...,...
41352,250001619358,2,https://whatsapp.com/dl/.,whatsapp.com,Whatsapp
38944,250001619382,6,https://api.whatsapp.com/send/?phone=551191410...,api.whatsapp.com,Whatsapp
39751,250001620641,6,https://wa.me/message/lcoz3opcshweo1,wa.me,Whatsapp
41085,250001639592,6,https://api.whatsapp.com/send/?phone=551999861...,api.whatsapp.com,Whatsapp


In [60]:
print(df[df.rede_social == 'Tinder'].reset_index(drop=True))

# 0  120001650955         3             https://tinder.com/@andrelacerdams   
# 1  130001615155         5  https://go.tinder.com/ilc4oml_lk4-m%c3%a9rcio

   SQ_CANDIDATO  NR_ORDEM                                         DS_URL  \
0  120001650955         3             https://tinder.com/@andrelacerdams   
1  130001615155         5  https://go.tinder.com/ilc4oml_lk4-m%c3%a9rcio   

          domain rede_social  
0     tinder.com      Tinder  
1  go.tinder.com      Tinder  


In [61]:
# exporta itens

In [63]:
valida_redes_rank.columns = ['Rede Social/Site', 'Registros de candidatos']

In [65]:
valida_redes_rank.to_csv('redes_sociais_candidatos.csv', index='False')